# Pandas Lab — FAANG-Level Hands-On

**Goal:** Build strong intuition for Pandas transformations used in ML pipelines (joins, groupby, leakage-safe features).

**Outcome:** You can write correct, scalable Pandas code and explain common gotchas (join explosion, leakage, apply misuse).


In [25]:
import pandas as pd
import numpy as np

def check(name: str, cond: bool):
    if not cond:
        raise AssertionError(f'Failed: {name}')
    print(f'OK: {name}')


## Section 0 — Synthetic Dataset (Interview-Friendly)
We’ll use synthetic tables to mirror typical ML feature engineering tasks.

In [26]:
rng = np.random.default_rng(0)

users = pd.DataFrame({
    'user_id': [101, 102, 103, 104],
    'country': ['IN', 'US', 'IN', 'CA'],
    'signup_ts': pd.to_datetime(['2025-01-01', '2025-01-03', '2025-01-04', '2025-01-10'])
})

events = pd.DataFrame({
    'user_id': rng.choice(users['user_id'], size=30, replace=True),
    'event_ts': pd.to_datetime('2025-01-01') + pd.to_timedelta(rng.integers(0, 20, size=30), unit='D'),
    'event': rng.choice(['view', 'add_to_cart', 'purchase'], size=30, replace=True, p=[0.7, 0.2, 0.1]),
    'amount': rng.choice([0, 0, 0, 19.99, 49.99, 99.99], size=30, replace=True),
})

# Ensure amount > 0 only for purchases (simple realism)
events.loc[events['event'] != 'purchase', 'amount'] = 0

users.head(), events.head()


(   user_id country  signup_ts
 0      101      IN 2025-01-01
 1      102      US 2025-01-03
 2      103      IN 2025-01-04
 3      104      CA 2025-01-10,
    user_id   event_ts        event  amount
 0      104 2025-01-17         view     0.0
 1      103 2025-01-04         view     0.0
 2      103 2025-01-02         view     0.0
 3      102 2025-01-18  add_to_cart     0.0
 4      102 2025-01-01         view     0.0)

## Section 1 — DataFrame Fundamentals

### Task 1.1: Basic filtering + new columns

Create:
- `purchases`: rows where `event == 'purchase'`
- add column `event_day` = date (no time)

# HINT:
- Use boolean indexing
- Use `.dt.floor('D')` or `.dt.date` (but prefer datetime64)

**Explain:** Why does vectorized `.dt` beat per-row parsing?


In [ ]:
# .dt is faster because it uses vectorized, C-level datetime operations instead of Python loops and function calls per row. Pandas stores datetimes as NumPy arrays, .dt accesses these arrays directly.

In [27]:
# TODO
purchases = events[events['event'] == 'purchase']
purchases = purchases.copy()
# purchases
purchases['event_day'] = purchases['event_ts'].dt.floor('D')

print(purchases)

check('purchases_only', set(purchases['event'].unique()) <= {'purchase'})
check('event_day_dtype', str(purchases['event_day'].dtype).startswith('datetime'))


   user_id   event_ts     event  amount  event_day
8      101 2025-01-10  purchase   99.99 2025-01-10
OK: purchases_only
OK: event_day_dtype


## Section 2 — GroupBy (Core for Features)

### Task 2.1: Per-user aggregates
Compute per-user features:
- `n_events`
- `n_purchases`
- `total_revenue` (sum of amount)

# HINT:
- Use `groupby('user_id').agg(...)`
- For `n_purchases`, use conditional aggregation

**FAANG gotcha:** Counting purchases using `amount>0` vs `event=='purchase'` can diverge if data is messy.


In [28]:
# TODO
user_features = (
    events.assign(is_purchase = (events['event'] == 'purchase').astype(int))
        .groupby('user_id', as_index=True)
        .agg(
            n_events = ('event', 'size'),
            n_purchases = ('is_purchase', 'sum'),
            total_revenue = ('amount', 'sum')
        )
)

user_features = user_features.reset_index()
print(user_features)
check('has_user_id', 'user_id' in user_features.columns)
check('has_total_revenue', 'total_revenue' in user_features.columns)


   user_id  n_events  n_purchases  total_revenue
0      101         6            1          99.99
1      102         4            0           0.00
2      103        12            0           0.00
3      104         8            0           0.00
OK: has_user_id
OK: has_total_revenue


### Task 2.2: GroupBy transform (row-level feature)

Add a column to `events` called `user_event_count` = number of events for that user.

# HINT:
- Use `groupby(...).transform('count')`

**Explain:** Why is `transform` different from `agg`?


In [ ]:
# 'transform' and 'agg' are both used with groupby, but they serve different purposes and return different shapes. 'agg' reduces groups, and 'transform' preserves the original row count. 

In [14]:
# TODO
events2 = events.copy()
events2['user_event_count'] = events.groupby('user_id')['event'].transform('size')
#print(events2)

check('user_event_count_nonnull', events2['user_event_count'].notna().all())
check('counts_positive', (events2['user_event_count'] > 0).all())


OK: user_event_count_nonnull
OK: counts_positive


## Section 3 — Joins & Merge (Feature Table Building)

### Task 3.1: Join user features back to users

Create `user_table` by joining `users` with `user_features` on `user_id`.

# HINT:
- Use `merge(..., how='left')`
- Fill missing aggregates with 0

**FAANG gotcha:** If you accidentally do a many-to-many join, row count explodes. Always validate row counts.


In [29]:
# TODO
user_table = users.merge(user_features, on='user_id', how='left')

# Fill NaNs for users with no events
for col in ['n_events', 'n_purchases', 'total_revenue']:
    if col in user_table.columns:
        user_table[col] = user_table[col].fillna(0)

check('rowcount_preserved', len(user_table) == len(users))
print(user_table)


OK: rowcount_preserved
   user_id country  signup_ts  n_events  n_purchases  total_revenue
0      101      IN 2025-01-01         6            1          99.99
1      102      US 2025-01-03         4            0           0.00
2      103      IN 2025-01-04        12            0           0.00
3      104      CA 2025-01-10         8            0           0.00


### Task 3.2: Join explosion debugging (mini)

Construct a tiny example where a join becomes many-to-many and explodes rows. Then fix it.

# HINT:
- Create `left` with duplicate keys
- Create `right` with duplicate keys
- Merge and observe rowcount

**Explain:** What join cardinality do you expect (1:1, 1:n, n:1, n:n)?


In [ ]:
# Join cardinality is the expected relationship between rows on the left and right tables for the join keys, i.e., how many rows can match per key on each side. This is expected to avoid explosions and leakage.
# 1:1 (one-to-one) - each key appears once in both tables.
# 1:n (one-to-many) - key is unique on the left, may repeat on the right
# n:1 (many-to-one) - key may repeat on the left, but is unique on the right
# n:n (many-to-many) - keys repeat on both sides

In [30]:
# TODO
left = pd.DataFrame({'k': [1, 1, 2], 'l': ['a', 'b', 'c']})
right = pd.DataFrame({'k': [1, 1, 2], 'r': ['x', 'y', 'z']})
exploded = left.merge(right, on='k', how='inner')
print('left rows', len(left), 'right rows', len(right), 'merged rows', len(exploded))

# TODO: Fix (example: deduplicate right to 1 row per k)
right_dedup = right.drop_duplicates('k', keep='first')
fixed = left.merge(right_dedup, on='k', how='inner')
print('fixed rows', len(fixed))


left rows 3 right rows 3 merged rows 5
fixed rows 3


## Section 4 — Time-based Features & Leakage (FAANG System Thinking)

### Task 4.1: Leakage-safe feature

For each user and each event row, compute `purchases_before` = number of purchases strictly BEFORE that `event_ts`.

Constraints:
- No Python loops over rows
- Must be time-aware

# HINT:
- Sort by user_id, event_ts
- Create boolean `is_purchase`
- Use groupby + cumsum, then shift

**FAANG gotcha:** If you include the current row in the count, you leak label info for purchase events.


In [31]:
# TODO
events3 = events.copy().sort_values(['user_id', 'event_ts']).reset_index(drop=True)
events3['is_purchase'] = (events3['event'] == 'purchase').astype(int)

# purchases up to and including current row
events3['purchases_cum'] = events3.groupby('user_id')['is_purchase'].cumsum()

# TODO: leakage-safe: purchases strictly before current row
events3['purchases_before'] = events3.groupby('user_id')['purchases_cum'].shift(1).fillna(0).astype(int)

check('nonnegative', (events3['purchases_before'] >= 0).all())
print(events3[['user_id','event_ts','event','purchases_before']].head(10))


OK: nonnegative
   user_id   event_ts        event  purchases_before
0      101 2025-01-02         view                 0
1      101 2025-01-06  add_to_cart                 0
2      101 2025-01-10     purchase                 0
3      101 2025-01-11         view                 1
4      101 2025-01-14  add_to_cart                 1
5      101 2025-01-20         view                 1
6      102 2025-01-01         view                 0
7      102 2025-01-16  add_to_cart                 0
8      102 2025-01-17         view                 0
9      102 2025-01-18  add_to_cart                 0


## Section 5 — Apply vs Vectorization

### Task 5.1: Remove slow apply

Create a `country_is_in` boolean column on `users`.
- First: do it with `.apply(...)` (slow style)
- Then: replace it with vectorized code

**Explain:** Why is `.apply` often slower in Pandas?


In [ ]:
# .apply() is often slower in pandas because it executes Python code row-by-row, while vectorized operations run compiled C/NumPy code on whole arrays at once.

In [32]:
import time
u = users.copy()

# TODO: slow style (keep for comparison)
t1 = time.time()
u['country_is_in_apply'] = u['country'].apply(lambda x: x == 'IN')
t_apply = time.time() - t1


# TODO: vectorized style
t2 = time.time()
u['country_is_in'] = u['country'] == 'IN'
t_vectorized = time.time() - t2

print('t_apply: ', t_apply)
print('t_vectorized: ', t_vectorized)
check('same_result', (u['country_is_in_apply'] == u['country_is_in']).all())
print(u)


t_apply:  0.0011799335479736328
t_vectorized:  0.0007810592651367188
OK: same_result
   user_id country  signup_ts  country_is_in_apply  country_is_in
0      101      IN 2025-01-01                 True           True
1      102      US 2025-01-03                False          False
2      103      IN 2025-01-04                 True           True
3      104      CA 2025-01-10                False          False


---
## Submission Checklist
- All TODOs completed
- All checks pass
- Explain prompts answered
